# Máster en Big Data y Data Science

# Metodologías de gestión y diseño de proyectos de big data

## Seminario III - Actividades de Preparación de los Datos y Modelado

---

### Sprint 2

### Carga de librerias a utilizar

In [4]:
# Se importan las librerias a utilizar
import pandas as pd
import numpy as np
import requests
import io

### Lectura del dataset generado en el Sprint 1

In [10]:
# Se hace la lectura del dataset del seminario y se obtiene su cabecera
#datos_completos = pd.read_csv('../datasets/datos_completos.csv', sep=',', index_col='Unnamed: 0')
url_datos_completos="https://raw.githubusercontent.com/JulietaNova/13MBID/main/datos_completos.csv"
datos_completos = pd.read_csv(url_datos_completos, sep=',')
datos_completos.head()

,Unnamed: 0,id_estudiante,propuesta,estado_inscripcion,fecha_ingreso,fecha_inscripcion,ingreso_aprobadas,ingreso_libres,ingreso_totales,cursadas_aprobadas,...,anio_ingreso,fecha_ultimo_examen,anio_ultima_reinscripcion,promedio_sin_aplazos,promedio_con_aplazos,actividades_aprobadas,total_actividades,regular,calidad,segundo_anio
0,0,CB-015876-0,134,Pendiente,01/04/2020,04/12/2019,2,0,2,0,...,2020,23/08/2021,2021.0,6.40,6.40,5,40,S,A,S
1,1,CA-016187-5,130,Pendiente,01/04/2020,09/12/2019,2,0,2,0,...,2020,14/12/2020,2021.0,6.50,6.50,2,42,S,A,N
2,2,CA-016558-7,130,Pendiente,01/04/2020,10/02/2020,2,0,2,0,...,2020,NaN,2021.0,8.17,8.17,6,42,S,A,S
3,3,CA-016164-6,130,Pendiente,01/04/2020,02/12/2019,0,2,2,0,...,2020,23/08/2021,2021.0,5.50,7.44,11,42,S,A,S
4,4,CA-016551-0,130,Aceptado,01/04/2020,10/02/2020,1,1,2,0,...,2020,NaN,NaN,0.00,0.00,0,42,N,P,N


### Revisión de meta-datos

In [11]:
def reporte_descripcion_dataset(df):
    columnas = df.columns
    print("Columnas del dataset:\n")
    for col in columnas:
        print(col)
    print(f"\nCantidad de filas: {df.shape[0]}")

print("Descripción del dataset 'datos_completos'")
reporte_descripcion_dataset(datos_completos)

Descripción del dataset 'datos_completos'
Columnas del dataset:

Unnamed: 0
id_estudiante
propuesta
estado_inscripcion
fecha_ingreso
fecha_inscripcion
ingreso_aprobadas
ingreso_libres
ingreso_totales
cursadas_aprobadas
cursadas_regulares
cursadas_libres
cursadas_totales
inscripciones_examenes
examenes_aprobados
anio_ingreso
fecha_ultimo_examen
anio_ultima_reinscripcion
promedio_sin_aplazos
promedio_con_aplazos
actividades_aprobadas
total_actividades
regular
calidad
segundo_anio

Cantidad de filas: 582


In [ ]:
import warnings
import sweetviz as sw

warnings.filterwarnings('ignore')

reporte_completo = sw.analyze(datos_completos)
reporte_completo.show_html(filepath='reporte_completo.html',
            open_browser=True,
            layout='vertical',
            scale=None)

                                             |      | [  0%]   00:00 -> (? left)

Report reporte_completo.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


### Realización de algunos ajustes sobre el dataset

In [14]:
limpiar_columnas = ['actividades_aprobadas', 'total_actividades', 'ingreso_aprobadas', 'ingreso_libres',
                    'ingreso_totales', 'cursadas_aprobadas', 'cursadas_regulares', 'cursadas_libres',
                    'cursadas_totales', 'fecha_ingreso', 'fecha_inscripcion', 'anio_ingreso',
                    'inscripciones_examenes', 'examenes_aprobados',
                    'promedio_sin_aplazos', 'promedio_con_aplazos', 'id_estudiante',
                   ]

# Generación de un atributo para el rendimiento en el ingreso
def pct_avance_ingreso(row):
    return row.ingreso_aprobadas / row.ingreso_totales

avance_ingreso = datos_completos.apply(lambda row: pct_avance_ingreso(row), axis=1).rename('avance_ingreso')
avance_ingreso

# Generación de un atributo nuevo para el rendimiento en el primer semestre
def pct_avance_semestre(row):
    return (row.cursadas_aprobadas + row.cursadas_regulares) / row.cursadas_totales

avance_1er_semestre = datos_completos.apply(lambda row: pct_avance_semestre(row), axis=1).rename('avance_1er_semestre')
avance_1er_semestre

# Atributo generado previamente pero que no se agregó
def pct_avance_carrera(row):
    return row.actividades_aprobadas / row.total_actividades

avance_carrera = datos_completos.apply(lambda row: pct_avance_carrera(row), axis=1).rename('avance_carrera')
avance_carrera

def pct_asignaturas_perdidas(row):
    return row.cursadas_libres / row.cursadas_totales

asignaturas_perdidas = datos_completos.apply(lambda row: pct_asignaturas_perdidas(row), axis=1).rename('asignaturas_perdidas')
asignaturas_perdidas

# Atributo generado previamente pero que no se agregó
def examenes_1er_semestre(row):
    if row.inscripciones_examenes > 0 and row.examenes_aprobados > 0:
        return 'A'
    elif row.inscripciones_examenes > 0:
        return 'I'
    else:
        return 'N'

examenes_1er_semestre = datos_completos.apply(lambda row: examenes_1er_semestre(row), axis=1).rename('examenes_1er_semestre')
examenes_1er_semestre

# Se pasan los valores de los promedios a rangos
valores = ['Bajo', 'Medio', 'Alto']

datos_completos['rango_promedios']=pd.cut(x=datos_completos['promedio_con_aplazos'],
                         bins=[0.0, 5.0, 7.0, 10.0],
                         include_lowest=True,
                         labels=valores)


# Se eliminan y agregan las columnas

temp = datos_completos.drop(limpiar_columnas, axis=1)
datos = pd.concat([temp, examenes_1er_semestre, avance_ingreso, avance_1er_semestre, avance_carrera,asignaturas_perdidas], axis=1)

datos.head(5)

,Unnamed: 0,propuesta,estado_inscripcion,fecha_ultimo_examen,anio_ultima_reinscripcion,regular,calidad,segundo_anio,rango_promedios,examenes_1er_semestre,avance_ingreso,avance_1er_semestre,avance_carrera,asignaturas_perdidas
0,0,134,Pendiente,23/08/2021,2021.0,S,A,S,Medio,N,1.0,0.0,0.125000,0.0
1,1,130,Pendiente,14/12/2020,2021.0,S,A,N,Medio,N,1.0,0.0,0.047619,0.0
2,2,130,Pendiente,NaN,2021.0,S,A,S,Alto,N,1.0,0.0,0.142857,0.0
3,3,130,Pendiente,23/08/2021,2021.0,S,A,S,Alto,N,0.0,0.0,0.261905,0.0
4,4,130,Aceptado,NaN,NaN,N,P,N,Bajo,N,0.5,0.0,0.000000,1.0


In [17]:
#pip install dtale

In [18]:
import dtale
import dtale.app as dtale_app

dtale_app.USE_COLAB = True

In [19]:
dtale.show(datos)

/usr/local/lib/python3.10/dist-packages/dtale/views.py:778: FutureWarning:

['fecha_ultimo_examen'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.



https://m3hcuewqyl-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/1

### Observaciones

En este punto se podrían aplicar transformaciones que fueran de utilidad para que la exportación de datos sea consistente con los requerimientos de las técnicas a aplicar en la fase siguiente. Ejemplos:
* binarización de atributos

In [15]:
#datos.to_csv('../datasets/datos_completos_filtrados.csv', sep=';', index=False)
from google.colab import files
datos.to_csv('datos_completos_filtrados.csv', sep=';', index=False)
files.download('datos_completos_filtrados.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>